In [2]:
%matplotlib inline
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
from scipy.stats import uniform as sp_randFloat
from sklearn import svm
from tqdm import tqdm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from time import time
from tabulate import tabulate
import numpy as np
import pandas as pd
import sklearn
import warnings
warnings.filterwarnings('ignore')
import os

In [3]:
from scipy.stats import randint as sp_randInt

from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.metrics import make_scorer
from scipy import sparse

In [4]:
import json
with open('GA_output_ET.json', 'r') as fp:
    feature_list = json.load(fp)

In [7]:
feature_list

{'SYN': ['Src Port',
  'Protocol',
  'Tot Fwd Pkts',
  'TotLen Fwd Pkts',
  'TotLen Bwd Pkts',
  'Bwd Pkt Len Max',
  'Flow Byts/s',
  'Flow Pkts/s',
  'Fwd IAT Std',
  'Fwd IAT Max',
  'Fwd IAT Min',
  'Bwd IAT Tot',
  'Bwd IAT Mean',
  'Bwd IAT Std',
  'Bwd IAT Max',
  'Bwd IAT Min',
  'Bwd PSH Flags',
  'Pkt Len Min',
  'Pkt Len Max',
  'Pkt Len Mean',
  'Pkt Len Std',
  'Pkt Len Var',
  'FIN Flag Cnt',
  'SYN Flag Cnt',
  'PSH Flag Cnt',
  'ACK Flag Cnt',
  'Pkt Size Avg',
  'Fwd Seg Size Avg',
  'Bwd Seg Size Avg',
  'Subflow Fwd Pkts',
  'Subflow Fwd Byts',
  'Subflow Bwd Pkts',
  'Active Mean',
  'Active Max',
  'Active Min',
  'Idle Std',
  'Label'],
 'HTTP': ['Dst Port',
  'Flow Duration',
  'Flow IAT Mean',
  'Fwd IAT Tot',
  'Fwd IAT Min',
  'Fwd Pkts/s',
  'FIN Flag Cnt',
  'SYN Flag Cnt',
  'Label'],
 'ACK': ['TotLen Bwd Pkts',
  'Fwd Pkt Len Max',
  'Fwd Pkt Len Mean',
  'Bwd Pkt Len Max',
  'Bwd Pkt Len Mean',
  'Bwd Pkt Len Std',
  'Flow Byts/s',
  'Flow IAT Std',
  'Fw

In [8]:
file_list={"SYN":[ './csvs\\dos-synflooding-1-dec.pcap_Flow.csv', './csvs\\dos-synflooding-2-dec.pcap_Flow.csv',],
"HTTP":['./csvs\\mirai-httpflooding-4-dec.pcap_Flow.csv', './csvs\\mirai-httpflooding-1-dec.pcap_Flow.csv'],
"ACK":['./csvs\\mirai-ackflooding-4-dec.pcap_Flow.csv','./csvs\\mirai-ackflooding-1-dec.pcap_Flow.csv'],
"UDP":[ './csvs\\mirai-udpflooding-4-dec.pcap_Flow.csv','./csvs\\mirai-udpflooding-1-dec.pcap_Flow.csv'],
"ARP":[ './csvs\\mitm-arpspoofing-6-dec.pcap_Flow.csv', './csvs\\mitm-arpspoofing-4-dec.pcap_Flow.csv'],
"SP":['./csvs\\scan-hostport-3-dec.pcap_Flow.csv','./csvs\\scan-hostport-4-dec.pcap_Flow.csv'],
"BF":[ './csvs\\mirai-hostbruteforce-5-dec.pcap_Flow.csv','./csvs\\mirai-hostbruteforce-3-dec.pcap_Flow.csv']}

In [9]:
def run_random_search(model, params, x_train, y_train):
    #grid = GridSearchCV(model, params, cv = ps, n_jobs = -1, scoring = score, verbose = 0, refit = False)
    grid =RandomizedSearchCV(model, param_grid, cv=ps,scoring = 'f1_macro')
    grid.fit(x_train, y_train)
    return (grid.best_params_, round(grid.best_score_,8),grid.best_estimator_)

In [10]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

# RandomizedSearchCV  DT

In [11]:
lines=[['criterion', 'max_depth', 'max_features', 'min_samples_split', "F1","Std","Time","No","Attack"]]

for j in file_list:
    print(j)
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    
    print ('%-35s %-20s %-8s %-8s' % ("HYPERPARAMETERS","F1 Score", "Time", "No"))

    nfolds=10
    param_grid = { 'criterion':['gini','entropy'],
                      "max_depth":np.linspace(1, 32, 32, endpoint=True).astype(int),
                     "min_samples_split": sp_randint(2,10),#uniform(0.1,1 ),
                        # "min_samples_leafs" : np.linspace(0.1, 0.5, 5, endpoint=True)
                        "max_features" : sp_randint(1,X_train.shape[1])}

    second=time()
    f1=[]
    clf=DecisionTreeClassifier()
    for ii in range(25):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))
    ######################################################################################################################
    for i in tqdm(range(50)):
        second=time()
        a,b,clf=run_random_search(DecisionTreeClassifier(),param_grid,X,y)
        f1=[]
        for ii in range(10):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:
results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("DT_HPO.csv",index=False)

final_parametres=[['criterion', 'max_depth', 'max_features', 'min_samples_split', "F1","Std","Time","No","Attack"]]
for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    m=df["max_depth"].min()
    df=df[df["max_depth"]==m]  
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))
    
    

SYN
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  2.387    24      


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [01:11<00:00,  1.43s/it]


HTTP
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  0.516    24      


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:12<00:00,  4.13it/s]


ACK
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.9168347420772845   0.29     24      


...

SP
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  0.262    24      


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:05<00:00,  8.35it/s]


BF
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  0.162    24      


100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [00:06<00:00,  7.74it/s]


    criterion      max_depth    max_features    min_samples_split        F1          Std    Time    No  Attack
--  -----------  -----------  --------------  -------------------  --------  -----------  ------  ----  --------
 0  gini                   4              30                    4  1         0             1.602    15  SYN
 1  gini                  16               5                    2  1         0             0.238    47  HTTP
 2  gini                  12              10                    2  0.916835  1.11022e-16   0.113    31  ACK
 3  entropy                3               6                    2  1         0             0.109    31  UDP
 4  gini                  25              23                    2  0.919672  0             0.224     1  ARP
 5  entropy                4              19                    7  1         0             0.108    20  SP
 6  entropy               18               2                    2  1         0             0.193     5  BF


# RandomizedSearchCV RF

In [12]:
lines=[['bootst', 'criter', 'max_depth', 'max_features',"min_samp_split","n_estimators", "F1","Std","Time","No","Attack"]]



for j in file_list:
    print(j)
    
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    
    
    
    
    print ('%-35s %-20s %-8s %-8s' % ("HYPERPARAMETERS","F1 Score", "Time", "No"))



    # use a full grid over all parameters
    param_grid = {"max_depth":np.linspace(1, 32, 32, endpoint=True).astype(int),
                  "n_estimators" : sp_randint(1, 200),
                  "max_features": sp_randint(1, 11),
                  "min_samples_split":sp_randint(2, 11),
                  "bootstrap": [True, False],
                  "criterion": ["gini", "entropy"]}
    second=time()
    f1=[]
    clf=RandomForestClassifier()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))


    
    ######################################################################################################################
    for i in tqdm(range(10)):
        second=time()
        a,b,clf=run_random_search(RandomForestClassifier(),param_grid,X,y)
        f1=[]
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:
results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("RF_HPO.csv",index=False)

final_parametres=[['bootst', 'criter', 'max_depth', 'max_features',"min_samp_split","n_estimators", "F1","Std","Time","No","Attack"]]

for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    m=df["max_depth"].min()
    df=df[df["max_depth"]==m]  
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    

SYN
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  1.655    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [05:46<00:00, 34.68s/it]


HTTP
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  0.712    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:05<00:00, 12.54s/it]


ACK
HYPERPARAMETERS                     F1 Score             Time     No      
default                             0.9168347420772845   0.298    0       


...

SP
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  0.273    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:33<00:00,  3.35s/it]


BF
HYPERPARAMETERS                     F1 Score             Time     No      
default                             1.0                  0.367    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:08<00:00,  6.83s/it]

    bootst    criter      max_depth    max_features    min_samp_split    n_estimators        F1          Std    Time    No  Attack
--  --------  --------  -----------  --------------  ----------------  --------------  --------  -----------  ------  ----  --------
 0  True      gini                4               9                 9             136  1         0            46.541     9  SYN
 1  False     entropy            22               1                 2              26  1         0            10.346     1  HTTP
 2  False     gini                8              10                 4              63  0.916835  1.11022e-16   4.077     3  ACK
 3  False     entropy             1               5                 7              59  1         0             2.625     9  UDP
 4  False     gini               30               8                 3              51  0.919672  0             4.522     2  ARP
 5  True      gini                5               4                 9             119  1       

# RandomizedSearchCV  KNeighborsClassifier

In [13]:
lines=[['algorithm', 'leaf_size', 'n_neighbors', 'weights', "F1","Std","Time","No","Attack"]]


for j in file_list:
    print(j)
    
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    

    # use a full grid over all parameters
    param_grid = {"n_neighbors" : sp_randint(1,64) ,  
                 "leaf_size": sp_randint(1,50) , 
                  "algorithm" : ["auto", "ball_tree", "kd_tree", "brute"],
                  "weights" : ["uniform", "distance"]}
    second=time()
    f1=[]
    clf=KNeighborsClassifier()
    for ii in range(25):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))


    
    ######################################################################################################################
    for i in tqdm(range(10)):
        second=time()
        a,b,clf=run_random_search(KNeighborsClassifier(),param_grid,X,y)
        f1=[]
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:
results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("knn_HPO.csv",index=False)

final_parametres=[['algorithm', 'leaf_size', 'n_neighbors', 'weights', "F1","Std","Time","No","Attack"]]

for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    m=df["n_neighbors"].min()
    df=df[df["n_neighbors"]==m]  
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))
    
    
    
    
    

SYN
default                             0.9544418091902468   128.226  24      


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [14:49<00:00, 88.96s/it]


HTTP
default                             0.9361026162449652   3.144    24      


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:20<00:00,  2.07s/it]


ACK
default                             0.41427005012531326  4.984    24      


...

SP
default                             0.9622386313788563   4.597    24      


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:21<00:00,  2.11s/it]


BF
default                             0.8737602932894387   1.744    24      


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:10<00:00,  1.07s/it]

    algorithm      leaf_size    n_neighbors  weights          F1          Std    Time    No  Attack
--  -----------  -----------  -------------  ---------  --------  -----------  ------  ----  --------
 0  kd_tree               10              9  distance   1         0            76.474     1  SYN
 1  auto                  43              7  distance   1         0             1.231     7  HTTP
 2  ball_tree             47             15  distance   0.916835  1.11022e-16   1.933     2  ACK
 3  kd_tree               34              3  distance   1         0             0.704     4  UDP
 4  kd_tree               19              4  distance   0.919672  0             0.697     4  ARP
 5  kd_tree               46              1  distance   1         0             2.508     3  SP
 6  kd_tree               21              2  distance   1         0             0.732     7  BF


# RandomizedSearchCV  SVM

In [14]:
lines=[['C', 'gamma', "F1","Std","Time","No","Attack"]]


for j in file_list:
    print(j)
    
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    


    param_grid =  {'C': [0.001, 0.01, 0.1, 1, 10], 'gamma' : [0.001, 0.01, 0.1, 1]}  

    second=time()
    f1=[]
    clf=svm.SVC()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))


    
    ######################################################################################################################
    for i in tqdm(range(1)):
        second=time()
        a,b,clf=run_random_search(svm.SVC(),param_grid,X,y)
        f1=[]
        for ii in range(1):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:

results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("svm_HPO.csv",index=False)


print (tabulate(results, headers=list(results.columns)))
    
    

SYN
default                             0.8633699337962215   0.7      0       


100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [17:40<00:00, 1060.24s/it]

{'gamma': 0.01, 'C': 1}                                                                    1.0                  1060.237 0       
HTTP


default                             0.6308547913760364   0.547    0       


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:15<00:00, 15.74s/it]

...

{'gamma': 0.01, 'C': 1}                                                                    0.9194952962115434   1.025    0       
SP
default                             0.7301892682181013   0.048    0       


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.02it/s]


{'gamma': 0.001, 'C': 1}                                                                   1.0                  0.973    0       
BF
default                             0.47279373977191136  0.103    0       


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.40s/it]

{'gamma': 1, 'C': 1}                                                                       1.0                  3.403    0       
        C    gamma        F1    Std      Time    No  Attack
--  -----  -------  --------  -----  --------  ----  --------
 0  0.01         1  1             0  1060.24      0  SYN
 1  0.1          1  1             0    15.739     0  HTTP
 2  1            1  0.916835      0     5.701     0  ACK
 3  0.1          1  1             0     1.689     0  UDP
 4  0.01         1  0.919495      0     1.025     0  ARP
 5  0.001        1  1             0     0.973     0  SP
 6  1            1  1             0     3.403     0  BF


In [15]:
from sklearn.naive_bayes import GaussianNB

In [16]:
lines=[['var_smoothing', "F1","Std","Time","No","Attack"]]
for j in file_list:
    print(j)
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    second=time()

    param_grid = {
        'var_smoothing': np.logspace(0,-9, num=100),
    }

 
    second=time()
    f1=[]
    clf=GaussianNB()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))
    ######################################################################################################################
    for i in tqdm(range(10)):
        second=time()
        a,b,clf=run_random_search(GaussianNB(),param_grid,X,y)
        f1=[]
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)


results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("NB_HPO.csv",index=False)
print (tabulate(results, headers=list(results.columns)))
    


SYN
default                             0.7158290715510804   0.064    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.10it/s]


HTTP
default                             0.7991613248350646   0.009    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  8.31it/s]


ACK
default                             0.7903516318707678   0.003    0       


...

SP
default                             0.9628631003738856   0.008    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 10.54it/s]


BF
default                             0.49909502262443445  0.015    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 12.42it/s]


      var_smoothing        F1          Std    Time    No  Attack
--  ---------------  --------  -----------  ------  ----  --------
 0      2.84804e-06  0.735984  0             0.924     0  SYN
 1      1e-05        0.735984  0             0.911     1  SYN
 2      1e-05        0.735984  0             0.816     2  SYN
 3      1.23285e-05  0.735984  0             0.892     3  SYN
 4      1.23285e-05  0.735984  0             0.93      4  SYN
 5      2.31013e-06  0.730786  0             0.917     5  SYN
 6      5.3367e-06   0.735984  0             0.877     6  SYN
 7      5.3367e-06   0.735984  0             0.893     7  SYN
 8      1.23285e-05  0.735984  0             0.882     8  SYN
 9      1e-05        0.735984  0             0.987     9  SYN
10      1e-09        0.799161  0             0.134     0  HTTP
11      1.51991e-07  0.726837  0             0.128     1  HTTP
12      2.31013e-08  0.730964  0             0.125     2  HTTP
13      1.87382e-09  0.800405  0             0.133     3  H

In [17]:

final_parametres=[["var_smoothing","F1","Std","Time","No","Attack"]]
for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))
    

      var_smoothing        F1          Std    Time    No  Attack
--  ---------------  --------  -----------  ------  ----  --------
 0      2.84804e-06  0.735984  0             0.924     0  SYN
 1      1.87382e-09  0.800405  0             0.133     3  HTTP
 2      1e-09        0.790352  0             0.094     8  ACK
 3      1e-09        0.637341  0             0.079     2  UDP
 4      1.23285e-09  0.49367   5.55112e-17   0.101     0  ARP
 5      1e-09        0.962863  1.11022e-16   0.05      1  SP
 6      0.432876     0.510061  0             0.067     5  BF


# LR

In [18]:
from scipy.stats import loguniform
from sklearn.linear_model import LogisticRegression
lines=[["C",'penalty','Solver',  "F1","Std","Time","No","Attack"]]
for j in file_list:
    print(j)
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    
    second=time()

    param_grid = {
     'solver': ['newton-cg', 'lbfgs', 'liblinear'],
    'penalty' : ['none', 'l1', 'l2', 'elasticnet'],
    'C' : loguniform(1e-5, 100)
        }
    

 
    second=time()
    f1=[]
    clf=LogisticRegression()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))
    ######################################################################################################################
    for i in tqdm(range(10)):
        second=time()
        a,b,clf=run_random_search(LogisticRegression(),param_grid,X,y)
        f1=[]
        for ii in range(5):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        #print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)


results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("LR_HPO.csv",index=False)
print (tabulate(results, headers=list(results.columns)))
    


SYN
default                             0.9131408605016138   0.941    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [11:46<00:00, 70.67s/it]


HTTP
default                             0.7624140350877193   0.092    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:12<00:00,  7.29s/it]


ACK
default                             0.8486774202372351   0.245    0       


...

SP
default                             0.8947933314531039   0.822    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [08:30<00:00, 51.03s/it]


BF
default                             0.437070938215103    0.016    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:22<00:00,  2.21s/it]

               C  penalty    Solver           F1          Std     Time    No  Attack
--  ------------  ---------  ---------  --------  -----------  -------  ----  --------
 0   1.09538e-05  none       lbfgs      0.913141  1.11022e-16   11.064     0  SYN
 1   8.20134e-05  none       lbfgs      0.913141  1.11022e-16    9.982     1  SYN
 2   0.678434     l1         liblinear  0.989245  0.00235819   150.755     2  SYN
 3   0.0637611    l2         lbfgs      0.918741  0              8.426     3  SYN
 4  14.6208       none       lbfgs      0.913141  1.11022e-16    7.244     4  SYN
 5   1.17201      l2         liblinear  0.918741  0              6.343     5  SYN
 6   1.0529       l1         liblinear  0.989273  0.00241489   101.864     6  SYN
 7  70.5022       l1         liblinear  0.997555  0.0048899     76.18      7  SYN
 8   0.00145617   l1         liblinear  0.912088  0.0133055    115.186     8  SYN
 9  23.8749       l1         liblinear  1         0            219.652     9  SYN
10   0.0

In [5]:
results

,C,penalty,Solver,F1,Std,Time,No,Attack
0,0.000011,none,lbfgs,0.913141,1.110000e-16,11.064,0,SYN
1,0.000082,none,lbfgs,0.913141,1.110000e-16,9.982,1,SYN
2,0.678434,l1,liblinear,0.989245,2.358190e-03,150.755,2,SYN
3,0.063761,l2,lbfgs,0.918741,0.000000e+00,8.426,3,SYN
4,14.620800,none,lbfgs,0.913141,1.110000e-16,7.244,4,SYN
...,...,...,...,...,...,...,...,...
65,44.113500,none,newton-cg,0.459043,5.550000e-17,3.402,5,BF
66,0.000039,l2,newton-cg,0.459043,5.550000e-17,2.831,6,BF
67,0.000416,l2,newton-cg,0.459043,5.550000e-17,3.073,7,BF
68,57.585200,none,newton-cg,0.459043,5.550000e-17,4.937,8,BF


In [6]:
final_parametres=[["C",'penalty','Solver',  "F1","Std","Time","No","Attack"]]
for i in results["Attack"].unique():
    df=results[results["Attack"]==i]
    m=df["F1"].max()
    df=df[df["F1"]==m]
    final_parametres.append(list(df.values)[0])
results = pd.DataFrame (final_parametres[1:], columns=  final_parametres[0])
print (tabulate(results, headers=list(results.columns)))
    

            C  penalty    Solver           F1         Std     Time    No  Attack
--  ---------  ---------  ---------  --------  ----------  -------  ----  --------
 0  23.8749    l1         liblinear  1         0           219.652     9  SYN
 1  33.7719    l1         liblinear  0.936259  0             4.542     1  HTTP
 2   4.7725    l1         liblinear  0.89331   0             2.746     6  ACK
 3   0.247099  l2         lbfgs      1         0             0.704     0  UDP
 4  10.1217    l1         liblinear  0.712696  0.00176454    5.112     0  ARP
 5   1.88657   l1         liblinear  1         0            55.614     1  SP
 6   0.144405  none       newton-cg  0.459043  5.55e-17      1.393     0  BF


# MLP

In [19]:
import scipy.stats
from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV


param_dist = [
    {'hidden_layer_sizes': (scipy.stats.randint(low=2**5, high=2**11+1).rvs(), )}
    for _ in range(100)] + [
    {'hidden_layer_sizes': (
            scipy.stats.randint(low=2 ** 5, high=2 ** 11 + 1).rvs(),
            scipy.stats.randint(low=2 ** 5, high=2 ** 11 + 1).rvs(),
    )} for i in range(100)
]

layers=[]

for i in param_dist:
    layers.append(i['hidden_layer_sizes'])



param_grid =  {
    'hidden_layer_sizes': layers,
    'solver': ['sgd', 'adam', 'lbfgs'],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001,0.001,0.01,0.1, 0.05],
    'learning_rate': ['constant','adaptive']
}


In [20]:
# MLP

import scipy.stats
from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV


param_dist = [
    {'hidden_layer_sizes': (scipy.stats.randint(low=2**5, high=2**11+1).rvs(), )}
    for _ in range(100)] + [
    {'hidden_layer_sizes': (
            scipy.stats.randint(low=2 ** 5, high=2 ** 11 + 1).rvs(),
            scipy.stats.randint(low=2 ** 5, high=2 ** 11 + 1).rvs(),
    )} for i in range(100)
]

layers=[]

for i in param_dist:
    layers.append(i['hidden_layer_sizes'])



param_grid =  {
    'hidden_layer_sizes': layers,
    'solver': ['sgd', 'adam', 'lbfgs'],
    'activation': ['relu', 'tanh'],
    'alpha': [0.0001,0.001,0.01,0.1, 0.05],
    'learning_rate': ['constant','adaptive']
}




lines=[['hidden_layer_sizes', 'solver','activation','alpha','learning_rate', "F1","Std","Time","No","Attack"]]


for j in file_list:
    print(j)
    
    
    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_train = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_train=df['Label'].cat.codes  


    df=pd.read_csv(file_list[j][0],usecols=feature_list[j]) 
    X_test = df.iloc[:,0:-1]
    df['Label'] = df['Label'].astype('category')
    y_test=df['Label'].cat.codes  
    
    X= np.concatenate([X_train, X_test])
    test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_test.shape[0])]
    y = np.concatenate([y_train, y_test])
    ps = PredefinedSplit(test_fold)
    

    param_grid =  {
        'hidden_layer_sizes': layers,
        'solver': ['sgd', 'adam', 'lbfgs'],
        'activation': ['relu', 'tanh'],
        'alpha': [0.0001,0.001,0.01,0.1, 0.05],
        'learning_rate': ['constant','adaptive']
    }



    second=time()
    f1=[]
    clf=MLPClassifier()
    for ii in range(1):
        clf.fit(X_train, y_train)
        predict =clf.predict(X_test)
        f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
    f1=sum(f1)/len(f1)   
    #if f1>0.76:
    print('%-35s %-20s %-8s %-8s' % ("default",f1,round(time()-second,3),ii))


    
    ######################################################################################################################
    for i in tqdm(range(1)):
        second=time()
        a,b,clf=run_random_search(MLPClassifier(),param_grid,X,y)
        f1=[]
        for ii in range(1):
            clf.fit(X_train, y_train)
            predict =clf.predict(X_test)
            f1.append(sklearn.metrics.f1_score(y_test, predict,average= "macro") )
        f1_result=sum(f1)/len(f1)   
        f1=np.array(f1)
        stndtd=f1.std()
        temp=list(a.values())
        print('%-90s %-20s %-8s %-8s' % (a,f1_result,round(time()-second,3),i))
        temp=temp+[f1_result,stndtd,round(time()-second,3),i,j]
        lines.append(temp)

        #if f1>0.76:

results = pd.DataFrame (lines[1:], columns = lines[0])
results.to_csv("ann_HPO.csv",index=False)


print (tabulate(results, headers=list(results.columns)))
    
    

SYN
default                             0.9414251832446034   5.965    0       


100%|████████████████████████████████████████████████████████████████████████████████| 1/1 [1:28:52<00:00, 5332.78s/it]

{'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': (725, 392), 'alpha': 0.01, 'activation': 'relu'} 0.9555925510375833   5332.765 0       
HTTP


default                             0.8790139259229572   0.502    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [34:42<00:00, 2082.62s/it]

...

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [16:27<00:00, 987.06s/it]

{'solver': 'adam', 'learning_rate': 'constant', 'hidden_layer_sizes': (288, 1882), 'alpha': 0.001, 'activation': 'tanh'} 0.98662454195854     987.058  0       
BF


default                             0.638276146244537    0.626    0       


100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [33:24<00:00, 2004.97s/it]

{'solver': 'lbfgs', 'learning_rate': 'adaptive', 'hidden_layer_sizes': (1993, 655), 'alpha': 0.01, 'activation': 'tanh'} 0.6305279359792713   2004.966 0       
    hidden_layer_sizes    solver    activation      alpha  learning_rate          F1    Std      Time    No  Attack
--  --------------------  --------  ------------  -------  ---------------  --------  -----  --------  ----  --------
 0  adam                  constant  (725, 392)     0.01    relu             0.955593      0  5332.77      0  SYN
 1  sgd                   constant  (1521, 218)    0.001   tanh             0.875596      0  2082.62      0  HTTP
 2  adam                  adaptive  (1919, 437)    0.001   tanh             0.896676      0  1009.69      0  ACK
 3  adam                  constant  (705, 151)     0.0001  tanh             0.972993      0  1145.94      0  UDP
 4  sgd                   constant  (883, 722)     0.05    tanh             0.761571      0   598.071     0  ARP
 5  adam                  constant  (288

-------------